Analyze the Elispot data from SDY196-SDY201


In [1]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", repos = "https://cloud.r-project.org")

Load necessary libraries


In [2]:
BiocManager::install("tidyverse")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'tidyverse'”


In [3]:
BiocManager::install("plotly")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'plotly'”


In [4]:
BiocManager::install("htmlwidgets")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'htmlwidgets'”


In [5]:
BiocManager::install("rstatix")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'rstatix'”


In [6]:
BiocManager::install("knitr")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'knitr'”


In [7]:
BiocManager::install("kableExtra")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'kableExtra'”


In [8]:
library(tidyverse)
library(plotly)
library(htmlwidgets)
library(rstatix)
library(knitr)
library(kableExtra)



── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout



Attaching package: ‘rstatix’


The foll

In [9]:
getwd()

[1] "/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/elispot/ipynb"

In [10]:
setwd("../data/SDY196-201")
getwd()

[1] "/Users/adeslatt/Scitechcon Dropbox/Anne DeslattesMays/projects/elispot/data/SDY196-201"

In [11]:
# Load data
elispot <- read.delim("elispot_result.txt", sep = "\t", stringsAsFactors = FALSE)
subjects <- read.delim("arm_2_subject.txt", sep = "\t", stringsAsFactors = FALSE)


In [12]:
# Merge ELISPOT results with subject metadata
elispot <- elispot %>% 
  left_join(subjects, by = "SUBJECT_ACCESSION") %>%
  mutate(
    STUDY = STUDY_ACCESSION,
    GROUP = str_to_title(SUBJECT_PHENOTYPE),
    AGE = MIN_SUBJECT_AGE_IN_YEARS,
    AGE_GROUP = case_when(
      AGE < 5 ~ "<5",
      AGE < 18 ~ "5-17",
      AGE < 40 ~ "18-39",
      AGE < 65 ~ "40-64",
      TRUE ~ "65+"
    )
  )

Warning message in left_join(., subjects, by = "SUBJECT_ACCESSION"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 2 of `x` matches multiple rows in `y`.
ℹ Row 35 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship = "many-to-many"` to silence this warning.”


In [13]:
# Summary stats table
summary_table <- elispot %>%
  group_by(STUDY, GROUP) %>%
  summarise(
    N = n(),
    Mean_Spot = mean(SPOT_NUMBER_REPORTED, na.rm = TRUE),
    Median_Spot = median(SPOT_NUMBER_REPORTED, na.rm = TRUE),
    SD_Spot = sd(SPOT_NUMBER_REPORTED, na.rm = TRUE),
    .groups = "drop"
  )


In [15]:
library(plotly)
library(htmlwidgets)


In [17]:
# Create summary table as a plotly object
summary_widget <- plot_ly(
  type = 'table',
  header = list(values = names(summary_table)),
  cells = list(values = lapply(summary_table, as.character))
)

# Save to HTML (NOT selfcontained)
htmlwidgets::saveWidget(summary_widget, "elispot_summary_table.html", selfcontained = FALSE)

In [ ]:


# Define age bins
elispot <- elispot %>% 
  mutate(age_group = cut(MIN_SUBJECT_AGE_IN_YEARS, breaks = c(0, 18, 30, 45, 60, Inf), 
                         labels = c("<18", "18-30", "31-45", "46-60", "60+"), right = FALSE))

# Summary statistics by study and group
summary_stats <- elispot %>% 
  group_by(study, group) %>% 
  summarise(across(where(is.numeric), list(mean = mean, sd = sd, median = median, n = length), 
                   .names = "{.col}_{.fn}"), .groups = "drop")

# Save summary table as HTML
summary_html <- kable(summary_stats, format = "html", table.attr = "class='table table-striped'", digits = 2) %>%
  kable_styling(bootstrap_options = c("striped", "hover", "condensed"))

save_html(summary_html, file = "elispot_summary_table.html")

# Wilcoxon test per study (example for 'spot_count')
tests <- elispot %>% 
  group_by(study) %>% 
  wilcox_test(spot_count ~ group) %>% 
  adjust_pvalue(method = "BH") %>% 
  add_significance()

# Merge back to annotate plots
test_labels <- tests %>% 
  mutate(y.position = max(elispot$spot_count, na.rm = TRUE) * 0.95) %>% 
  select(study, p.adj, p.adj.signif, y.position)

# Interactive box plot by group
p <- elispot %>% 
  plot_ly(x = ~group, y = ~spot_count, color = ~group, type = "box", 
          hoverinfo = 'text',
          text = ~paste("Study:", study, "<br>Age:", MIN_SUBJECT_AGE_IN_YEARS, "<br>Subject:", SUBJECT_ACCESSION)) %>%
  layout(title = "ELISPOT Spot Count by Group",
         yaxis = list(title = "Spot Count"),
         boxmode = "group")

# Save interactive plot
saveWidget(p, "elispot_group_boxplot.html")

# Interactive box plot by group and age group
p_age <- elispot %>% 
  plot_ly(x = ~age_group, y = ~spot_count, color = ~group, type = "box",
          hoverinfo = 'text',
          text = ~paste("Study:", study, "<br>Subject:", SUBJECT_ACCESSION)) %>%
  layout(title = "ELISPOT Spot Count by Age Group and Group",
         yaxis = list(title = "Spot Count"),
         boxmode = "group")

# Save age group box plot
saveWidget(p_age, "elispot_agegroup_boxplot.html")